In [2]:
# ## 1. Carga del Dataset
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import pandas as pd

# Transformaciones: Normalización y conversión a tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # mean y std de MNIST
])

# Descarga y creación de dataloaders
train_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# Para experimentación, los batch_size variarán por configuración
# DataLoader de ejemplo (se redefinirá según configuración)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


# ## 2. Definición de la MLP flexible
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, activation_fn):
        super().__init__()
        layers = []
        in_features = input_size
        # Capas ocultas
        for h in hidden_layers:
            layers.append(nn.Linear(in_features, h))
            layers.append(activation_fn())
            in_features = h
        # Capa de salida
        layers.append(nn.Linear(in_features, output_size))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # aplanar imágenes 28x28
        return self.net(x)


def train_and_evaluate(config, verbose=True):
    """
    Entrena y evalúa un modelo MLP según la configuración dada.
    config: dict con keys:
      'hidden_layers', 'activation', 'lr', 'batch_size', 'epochs'
    Devuelve: accuracy en test
    """
    # Selección de función de activación
    activations = {
        'relu': nn.ReLU,
        'tanh': nn.Tanh,
        'sigmoid': nn.Sigmoid,
    }
    act_fn = activations[config['activation']]

    # Dataloaders según batch_size
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    test_loader  = DataLoader(test_dataset,  batch_size=config['batch_size'], shuffle=False)

    # Modelo, optimizador y criterio
    model = MLP(784, config['hidden_layers'], 10, act_fn)
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    criterion = nn.CrossEntropyLoss()

    # Loop de entrenamiento
    for epoch in range(1, config['epochs'] + 1):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            logits = model(X_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()
        if verbose:
            print(f"Epoca {epoch}/{config['epochs']}, Loss: {loss.item():.4f}")

    # Evaluación
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            logits = model(X_batch)
            preds = logits.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    accuracy = correct / total
    if verbose:
        print(f"Accuracy en test: {accuracy:.4f}\n")
    return accuracy

# ## 3. Configuraciones iniciales
configs = [
    {
        'name': 'MLP_ReLU_1hidden',
        'hidden_layers': [128],
        'activation': 'relu',
        'lr': 1e-3,
        'batch_size': 64,
        'epochs': 5
    },
    {
        'name': 'MLP_Tanh_2hidden',
        'hidden_layers': [256, 128],
        'activation': 'tanh',
        'lr': 5e-4,
        'batch_size': 128,
        'epochs': 7
    },
    {
        'name': 'MLP_Sigmoid_3hidden',
        'hidden_layers': [512, 256, 128],
        'activation': 'sigmoid',
        'lr': 1e-4,
        'batch_size': 256,
        'epochs': 10
    }
]

# Ejecutar entrenamientos
results = []
for cfg in configs:
    print(f"=== Configuración: {cfg['name']} ===")
    acc = train_and_evaluate(cfg)
    results.append({'Model': cfg['name'], 'Accuracy': acc})

# ## 4. Tuning de Hiperparámetros (Grid Search)
from itertools import product

grid = {
    'hidden_layers': [[256], [256, 128]],
    'activation': ['relu', 'tanh'],
    'lr': [1e-3, 5e-4],
    'batch_size': [64, 128],
    'epochs': [5]
}

best = {'config': None, 'accuracy': 0}
for hl, act, lr, bs, ep in product(grid['hidden_layers'], grid['activation'], grid['lr'], grid['batch_size'], grid['epochs']):
    cfg = {'name': 'GS', 'hidden_layers': hl, 'activation': act, 'lr': lr, 'batch_size': bs, 'epochs': ep}
    print(f"Probando: layers={hl}, act={act}, lr={lr}, batch_size={bs}")
    acc = train_and_evaluate(cfg, verbose=False)
    if acc > best['accuracy']:
        best = {'config': cfg, 'accuracy': acc}

print("\nMejor configuración grid search:", best)

# ## 5. Evaluación comparativa y reporte
results.append({'Model': 'GridSearchBest', 'Accuracy': best['accuracy']})

df_results = pd.DataFrame(results).sort_values(by='Accuracy', ascending=False)
print("Ranking de modelos:\n", df_results)

# Conclusiones (imprimir manualmente al finalizar)
print("\n-- ¿Qué hiperparámetros influyeron más? --")
print("Generalmente el número de neuronas y la elección de la función de activación tienen gran impacto,")
print("así como la tasa de aprendizaje. Batch size y epochs afectan estabilidad y convergencia.")


100%|██████████| 9.91M/9.91M [00:03<00:00, 2.77MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 86.1kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 1.74MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.05MB/s]


=== Configuración: MLP_ReLU_1hidden ===
Epoca 1/5, Loss: 0.0473
Epoca 2/5, Loss: 0.0824
Epoca 3/5, Loss: 0.0689
Epoca 4/5, Loss: 0.1716
Epoca 5/5, Loss: 0.0095
Accuracy en test: 0.9739

=== Configuración: MLP_Tanh_2hidden ===
Epoca 1/7, Loss: 0.2780
Epoca 2/7, Loss: 0.0926
Epoca 3/7, Loss: 0.1869
Epoca 4/7, Loss: 0.0258
Epoca 5/7, Loss: 0.1125
Epoca 6/7, Loss: 0.0327
Epoca 7/7, Loss: 0.0385
Accuracy en test: 0.9815

=== Configuración: MLP_Sigmoid_3hidden ===
Epoca 1/10, Loss: 1.7137
Epoca 2/10, Loss: 1.1876
Epoca 3/10, Loss: 0.6652
Epoca 4/10, Loss: 0.5871
Epoca 5/10, Loss: 0.3836
Epoca 6/10, Loss: 0.3506
Epoca 7/10, Loss: 0.2259
Epoca 8/10, Loss: 0.2991
Epoca 9/10, Loss: 0.1763
Epoca 10/10, Loss: 0.1871
Accuracy en test: 0.9426

Probando: layers=[256], act=relu, lr=0.001, batch_size=64
Probando: layers=[256], act=relu, lr=0.001, batch_size=128
Probando: layers=[256], act=relu, lr=0.0005, batch_size=64
Probando: layers=[256], act=relu, lr=0.0005, batch_size=128
Probando: layers=[256], 

# Comparación de resultados

La red con **Tanh y dos capas ocultas (256 → 128)** obtuvo el mejor accuracy (98.15 %), seguida muy de cerca por la mejor configuración del Grid Search (ReLU, [256], lr = 5e-4, bs = 64) con 98.11 %. La MLP con ReLU y una sola capa (128) quedó en 97.39 %, y la de tres capas con Sigmoid en 94.26 %.

| Posición | Modelo                  | Accuracy |
|:--------:|:-----------------------:|:--------:|
| 1        | MLP_Tanh_2hidden        | 98.15 %  |
| 2        | GridSearchBest          | 98.11 %  |
| 3        | MLP_ReLU_1hidden        | 97.39 %  |
| 4        | MLP_Sigmoid_3hidden     | 94.26 %  |

---

## ¿Qué hiperparámetros influyeron más en la mejora del rendimiento? ¿Por qué?

- **Función de activación**  
  - **Tanh** superó a ReLU gracias a centrar sus salidas en cero, mejorando la dinámica del gradiente.  
  - **Sigmoid** presentó saturación y gradientes muy pequeños en redes profundas, enlenteciendo la convergencia.

- **Arquitectura (número y tamaño de capas ocultas)**  
  - Dos capas con suficiente ancho (256 → 128) modelaron mejor las complejidades del MNIST sin incurrir en los problemas de entrenamiento de redes muy profundas.

- **Tasa de aprendizaje (learning rate)**  
  - Un **lr = 5e-4** logró un balance óptimo entre velocidad de convergencia y estabilidad, evitando oscilaciones.

- **Batch size**  
  - Un **batch size pequeño (64)** introdujo ruido en el gradiente que actuó como regularizador, mejorando la generalización frente a bs = 128.

En conjunto, la **elección de la activación** y la **arquitectura de la red** fueron los factores clave, seguidos por la **tasa de aprendizaje** y el **tamaño de lote**, ya que determinan cómo y con qué rapidez el modelo explora el espacio de parámetros durante el entrenamiento.  
